# Apprentissage supervisé - Application

Nous allons travailler sur un jeu de données `Adult`, dont toutes les informations sont sur [cette page](https://archive.ics.uci.edu/dataset/2/adult). Celui-ci provient d'une enquête auprès d'états-uniens, sur leur salaire (binaire - plus ou moins de 50K$) et un certain nombres de critères (âge, sexe, CSP, niveau d'études, ...)

Pour l'importer, il vous faire les 2 opérations suivantes :

- Installer le package `ucimlrepo`
- Lancer le code ci-dessous

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id = 2)
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [2]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [3]:
y.head()

,income
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K


### Informations sur la variable à prédire

On remarque qu'il semble y avoir un problème d'importation... Un "." étant présent sur certaines modalités.

In [4]:
import pandas

pandas.crosstab(y.income, "Nb")

col_0,Nb
income,
<=50K,24720
<=50K.,12435
>50K,7841
>50K.,3846


In [5]:
y = y.assign(income = y.income.str.replace(".", ""))
pandas.crosstab(y.income, "Nb")

col_0,Nb
income,
<=50K,37155
>50K,11687


## Création de 2 jeux de données

Il est important, dans un cadre supervisé, de créer 2 jeux de données (un d'apprentissage et un de test) pour paramètrer les méthodes, dans un rapport 2 tiers / 1 tiers (voire jusque 80% - 20%). Cela permet d'éviter le sur-apprentissage particulièrement. On peut même envisager de découper en 3 jeux de données, le dernier étant de validation pour avir une estimation de la performance des modèles sur des données nouvelles.

On va donc créer un jeu d'apprentissage avec 75% des réponses, et un jeu de données test avec 25%.

In [6]:
X_train = X.sample(frac = .8)
X_test = X.drop(X_train.index)

In [7]:
y_train = y.iloc[X_train.index]
y_test = y.iloc[X_test.index]

In [8]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
13343,56,Private,208451,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
31586,49,Private,104542,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States
29245,58,Local-gov,32855,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,48,United-States
24672,28,Private,293926,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1740,30,United-States
9873,47,Local-gov,285060,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,41,United-States


In [9]:
y_train.head()

,income
13343,>50K
31586,<=50K
29245,<=50K
24672,<=50K
9873,>50K


In [10]:
X_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
10,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States
14,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?
15,34,Private,245487,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,Amer-Indian-Eskimo,Male,0,0,45,Mexico
22,35,Federal-gov,76845,9th,5,Married-civ-spouse,Farming-fishing,Husband,Black,Male,0,0,40,United-States
23,43,Private,117037,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,2042,40,United-States


In [11]:
y_test.head()

,income
10,>50K
14,>50K
15,<=50K
22,<=50K
23,<=50K


A partir de maintenant, nous allons donc créer nos modèles sur le jeu `train` et chercher nos paramètres avec le jeu `test`.

## A FAIRE

- Encoder les variables qualitatives en *one-hot* pour les utiliser dans les modèles
- Réaliser le modèle de regression logistique complet
- Chercher un nombre de voisins intéressants pour la méthode des $k$ plus proches voisins
    - Jeu de données `test` à utiliser avec les fonctions `predict()` et `predict_proba()`
- Chercher l'arbre CART élagué le plus intéressant
    - Jeu de données `test` à utiliser pour l'élagage (fonction `cost_complexity_pruning_path()` à appliquer sur les 2 - `train` et `test`)
- Chercher un MLP intéressant
    - Jeu de données `test` à utiliser avec les fonctions `predict()` et `predict_proba()`
- Comparer les modèles sur leurs résultats sur les 2 jeux de données (comparaison de courbes *ROC* et *AUC*)